# Technology

gdsfactory has a default `config.yml` file that inclues all the defaults for the generic Technology.

[YAML](https://www.cloudbees.com/blog/yaml-tutorial-everything-you-need-get-started/) is a human friendly format

To overwrite any of the defaults you can just create a `config.yml` in the current directory of your project.

The `conig.yml` can define:

- Layers



## Layers

A GDS has different layers to describe the different fabrication process steps.

GDS layers have 2 integer numbers: GDSlayer, GDSpurpose

Klayout shows Layers with a color, style and transparency when showing GDS layouts.

In [ ]:
import omegaconf
import dataclasses
import pp
from pp.layers import preview_layerset

In [ ]:
layer_wg = pp.LAYER.WG
print(layer_wg)

In [ ]:
c = preview_layerset()
c

## TECH

As you can see in `pp.tech.TECH` you can define all your technology constants in dataclasses. 
This is a convenient way to group all the specific constants for a technology. 

In [ ]:
import pp

pp.TECH.waveguide.strip

In [ ]:
import dataclasses
from pp.tech import Waveguide, Section, TECH

waveguide_pn = Waveguide(
    width=0.5,
    layer=pp.LAYER.WG,
    sections=(
        Section(width=2, layer=pp.LAYER.N, offset=+1),
        Section(width=2, layer=pp.LAYER.P, offset=-1),
    ))


TECH.waveguide.pn = waveguide_pn

In [ ]:
waveguide_pn_settings = pp.components.straight(waveguide='pn')
waveguide_pn_settings

## PDK

How can you customize components for a particular technology?

You can customize the `TECH` as you have done before to customize any gdsfactory defaults.

In [ ]:
import pp

c = pp.components.straight(waveguide='nitride')
c

In [ ]:
wg = pp.components.straight(waveguide='nitride')
gc = pp.components.grating_coupler_elliptical_te(layer=pp.TECH.waveguide.nitride.layer, wg_width=pp.TECH.waveguide.nitride.width)
wg_gc = pp.routing.add_fiber_single(component=wg, grating_coupler=gc, waveguide='nitride')
wg_gc

In [ ]:
## Fab A

"""Lets for example customize the default gdsfactory PDK

Fab A is mostly defined using Metal layers in GDS layer (30, 0)

The metal layer traces are 2um wide

"""
import dataclasses
import pp
from pp.tech import TECH, Layer, Waveguide


@dataclasses.dataclass
class Metal1(Waveguide):
    width: float = 2.0
    width_wide: float = 10.0
    auto_widen: bool = False
    layer: Layer = (30, 0)
    radius: float = 10.0


METAL1 = Metal1()

TECH.waveguide.metal1 = METAL1


wg = pp.components.straight(length=20, waveguide="metal1")
gc = pp.components.grating_coupler_elliptical_te(
    layer=METAL1.layer, wg_width=METAL1.width
)

wg_gc = pp.routing.add_fiber_array(
    component=wg, grating_coupler=gc, waveguide="metal1"
)
wg_gc

In [ ]:
coupler = pp.components.coupler(cross_section_name='metal1')
coupler

## Fab B

Lets for example customize the default gdsfactory PDK for a particular fab

Fab B is mostly uses optical layers but the waveguides required many cladding layers to avoid tiling, dopants...

Lets say that the waveguides are defined in layer (2, 0) and are 0.3um wide


In [ ]:
from typing import Tuple
import dataclasses
from pp.tech import LAYER, TECH, Layer, Waveguide
import pp


@dataclasses.dataclass
class StripB(Waveguide):
    width: float = 0.3
    width_wide: float = 10.0
    auto_widen: bool = False
    layer: Layer = (2, 0)
    radius: float = 10.0
    min_spacing: float = 10.0
    layers_cladding: Tuple[Layer] = ((71, 0), (68, 0))


STRIPB = StripB()
TECH.waveguide.stripb = STRIPB
wg = pp.components.straight(length=20, cross_section_name="stripb")
gc = pp.components.grating_coupler_elliptical_te(layer=STRIPB.layer, wg_width=STRIPB.width)
wg_gc = pp.routing.add_fiber_array(
    component=wg, grating_coupler=gc, cross_section_name="stripb"
)
wg_gc

## Fab C

Lets assume that fab C has both Silicon and Silicon Nitride components, and you need different waveguide widths for C and O band.

Lets asume that O band nitride waveguide width is 0.9 and Cband Nitride waveguide width is 1um.

Lets also that this foundry has an LVS flow where all components have optical pins defined in layer (100, 0)

In [ ]:
from typing import Callable, Optional, Dict, Tuple
import pp
from pp.cell import cell
from pp.component import Component, ComponentReference
from pp.pdk import Pdk
from pp.tech import TECH
from pp.add_pins import add_pins, add_pin_square_inside

In [ ]:
WIDTH_NITRIDE_OBAND = 0.9
WIDTH_NITRIDE_CBAND = 1.0
PORT_TYPE_TO_LAYER = dict(optical=(100, 0))

                    
def add_pins_custom(
    component: Component,
    reference: Optional[ComponentReference] = None,
    function: Callable = add_pin_square_inside,
    port_type_to_layer: Dict[str, Tuple[int, int]] = PORT_TYPE_TO_LAYER,
    **kwargs,
) -> None:
    """Add Pin port markers.

    Args:
        component: to add ports
        function:
        port_type_to_layer: dict mapping port types to marker layers for ports

    """
    reference = reference or component
    for p in reference.ports.values():
        layer = port_type_to_layer[p.port_type]
        function(component=component, port=p, layer=layer, label_layer=layer, **kwargs)


@cell
def mmi1x2_nitride_cband(
    width: float = WIDTH_NITRIDE_CBAND,
    width_taper: float = 1.0,
    length_taper: float = 10.0,
    length_mmi: float = 5.5,
    width_mmi: float = 2.5,
    gap_mmi: float = 0.25,
    with_cladding_box: bool = True,
    waveguide: str = "nitride",
    **kwargs

)->Component:
    c= pp.components.mmi1x2(
        width=width,
        width_taper=width_taper,
        length_taper=length_taper,
        length_mmi = length_mmi,
        width_mmi = width_mmi,
        gap_mmi  = gap_mmi,
        with_cladding_box = with_cladding_box,
        waveguide = waveguide,
        **kwargs
    )
    add_pins_custom(c)
    return c

@cell
def mzi_nitride_cband(delta_length:float=10.)->Component:
    c = pp.c.mzi(delta_length=delta_length, splitter=mmi1x2_nitride_cband, waveguide='nitride', width=WIDTH_NITRIDE_CBAND)
    add_pins_custom(c)
    return c


c = mzi_nitride_cband()
c

In [ ]:
TECH.waveguide.metal_routing

In [ ]:
TECH.waveguide

In [ ]:
TECH.waveguide.nitride